# Data Owner

In [1]:
import syft as sy
import tenseal as ts
import torch
import pandas as pd
import random
import numpy as np

import pytest
from time import time
import matplotlib.pyplot as plt

### Joining the Duet

In [3]:
duet = sy.duet("efec618a93e5d4d007bb62f93814aa03")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: 8d0966df25e61f47b3119c7102f6780f

♫♫♫ > ...waiting for partner to connect...

♫♫♫ > CONNECTED!


### Defining The Model

Here we define the models used by the Data Scientist.

 - __LR__ is a simple logistic regression for plain data.
 - __EncryptedLR__ adapts the forward method to the API exposed by the encrypted ciphertexts.

In [4]:
class LR(torch.nn.Module):

    def __init__(self, n_features):
        super(LR, self).__init__()
        self.lr = torch.nn.Linear(n_features, 1)
        
    def forward(self, x):
        out = torch.sigmoid(self.lr(x))
        return out
    
class EncryptedLR:
    def __init__(self, torch_lr):
        # TenSEAL processes lists and not torch tensors
        # so we take out parameters from the PyTorch model
        self.weight = torch_lr.lr.weight.data.tolist()[0]
        self.bias = torch_lr.lr.bias.data.tolist()
        
    def forward(self, enc_x):
        # We don't need to perform sigmoid as this model
        # will only be used for evaluation, and the label
        # can be deduced without applying sigmoid
        enc_out = enc_x.dot(self.weight) + self.bias
        return enc_out
    
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

### Look at the data available in Duet

In [5]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: befbda91a96b438b94903fe07bf33d2b>,[x_train],,<class 'torch.Tensor'>
1,<UID: 820c2aad01bf4c1986e7ebf32bcbc3fa>,[y_train],,<class 'torch.Tensor'>


### Request the training data

We request access to the training data.

In [6]:
x_train_ptr = duet.store["x_train"]
y_train_ptr = duet.store["y_train"]
x_train_ptr.request(reason="I would like to get the training data")
y_train_ptr.request(reason="I would like to get the training labels")


In [7]:
duet.requests.pandas

,Requested Object's tags,Reason,Request ID,Requested Object's ID,Requested Object's type
0,[x_train],I would like to get the training data,<UID: 90854230809541608a55bed379ec17f7>,<UID: befbda91a96b438b94903fe07bf33d2b>,<class 'torch.Tensor'>
1,[y_train],I would like to get the training labels,<UID: a6b07546455f4b3a978323addf9c47c1>,<UID: 820c2aad01bf4c1986e7ebf32bcbc3fa>,<class 'torch.Tensor'>


### Train the logistic regression

Now we train our Logistic regression with the data from the DO instance.

In [8]:
x_train = x_train_ptr.get(delete_obj=False)
y_train = y_train_ptr.get(delete_obj=False)

n_features = x_train.shape[1]
model = LR(n_features)

# use gradient descent with a learning_rate=1
optim = torch.optim.SGD(model.parameters(), lr=1)

# use Binary Cross Entropy Loss
criterion = torch.nn.BCELoss()

In [10]:
EPOCHS = 20

def train(model, optim, criterion, x, y, epochs=EPOCHS):
    for e in range(1, epochs + 1):
        optim.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optim.step()
        print(f"Loss at epoch {e}: {loss.data}")
    return model

model = train(model, optim, criterion, x_train, y_train)

Loss at epoch 1: 0.6129701137542725
Loss at epoch 2: 0.6129413843154907
Loss at epoch 3: 0.6129173040390015
Loss at epoch 4: 0.612896740436554
Loss at epoch 5: 0.61287921667099
Loss at epoch 6: 0.6128641963005066
Loss at epoch 7: 0.6128511428833008
Loss at epoch 8: 0.6128397583961487
Loss at epoch 9: 0.6128299236297607
Loss at epoch 10: 0.6128214001655579
Loss at epoch 11: 0.6128137707710266
Loss at epoch 12: 0.612807035446167
Loss at epoch 13: 0.6128012537956238
Loss at epoch 14: 0.6127961277961731
Loss at epoch 15: 0.6127914786338806
Loss at epoch 16: 0.6127874851226807
Loss at epoch 17: 0.6127839088439941
Loss at epoch 18: 0.6127808094024658
Loss at epoch 19: 0.6127779483795166
Loss at epoch 20: 0.6127755045890808
Loss at epoch 21: 0.6127732992172241
Loss at epoch 22: 0.6127712726593018
Loss at epoch 23: 0.612769603729248
Loss at epoch 24: 0.6127680540084839
Loss at epoch 25: 0.612766683101654
Loss at epoch 26: 0.6127654910087585
Loss at epoch 27: 0.6127643585205078
Loss at epoch 28

### Request the encrypted data

We create the model variant that cand handle encrypted tensors.

In [11]:
eelr = EncryptedLR(model)

### Look at the Duet store


In [12]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: befbda91a96b438b94903fe07bf33d2b>,[x_train],,<class 'torch.Tensor'>
1,<UID: 820c2aad01bf4c1986e7ebf32bcbc3fa>,[y_train],,<class 'torch.Tensor'>


### Get pointers to the encrypted vectors and request permission to work on them.

In [14]:
ctx_ptr = duet.store["context"]
enc_x_test_ptr = duet.store["enc_x_test"]

ctx_ptr.request(reason="I would like to get the context")
enc_x_test_ptr.request(reason="I would like to get encrypted test set")

### Encrypted evaluation

In [16]:
def encrypted_evaluation(model, enc_x_test):
    results = []
    
    correct = 0
    for enc_x in enc_x_test:
        # encrypted evaluation
        results.append(model(enc_x))
    return results

enc_x_test = enc_x_test_ptr.get(delete_obj=False)
ctx = ctx_ptr.get(delete_obj=False)
for t in enc_x_test:
    t.link_context(ctx)

eval_res = encrypted_evaluation(eelr, enc_x_test)

### Send the result to duet store and print it 

In [17]:
result_eval_ptr = sy.lib.python.List(eval_res).send(duet, pointable=True, tags=["result_eval"])

print(duet.store)

[<syft.proxy.torch.TensorPointer object at 0x7f3ee0d4a510>, <syft.proxy.torch.TensorPointer object at 0x7f3ee0d4a7d0>, <syft.proxy.tenseal.ContextPointer object at 0x7f3ee0d4aa50>, <syft.proxy.syft.lib.python.ListPointer object at 0x7f3ee2488e90>, <syft.proxy.syft.lib.python.ListPointer object at 0x7f3ee24888d0>]
